# Top skills required for Job

In [ ]:
#importing Libraries
import seaborn as sns
from datasets import load_dataset
import pandas as pd
import ast
import matplotlib.pyplot as plt

In [ ]:
#load and clean data

#loading of data
data = load_dataset('lukebarousse/data_jobs')
df = data['train'].to_pandas()

#data cleaning
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)